In [ ]:
# Comenzaremos copiando el codigo de la función nextpow2 que realizó en el ejercicio anterior

def nextpow2(N):
    Nout = None  # aqui el resultado

    # YOUR CODE HERE
    Nout = 2 ** int(np.ceil(np.log2(N)))

    Nout = int(Nout)  # el resultado debe ser entero
    return Nout

def determina_frecuencia_fund(x: np.ndarray, fmin: float, fmax: float, fs: float) -> float:
    ''' Devuelve la frecuencia fundamental de un sonido musical en Hz. '''
    freq_fund = None  # aqui el resultado

    # Determina N y calcula modulo de la FFT
    N = nextpow2(len(x))          # aqui el valor de N
    X = np.fft.fft(x, N)
    absX = np.abs(X)              # aqui el resultado del modulo de la FFT

    assert N == X.shape[0], "toma la FFT de N puntos"
    assert np.log2(N).is_integer(), "N debe ser potencia de 2"

    # Determina kmin y kmax (índices del rango [fmin, fmax])
    kmin = int(round(fmin / fs * N))
    kmax = int(round(fmax / fs * N))

    # Hacer bucle entre kmin y kmax y sumar los valores de los 5 armónicos
    # para cada k. Si el valor de la suma es mas grande hasta el momento
    # guarda el valor de k y el valor de la suma.

    max_absX = 0   # aqui el valor maximo de la suma de los 5 armonicos
    k_best = 0     # aqui el valor de k que la suma de los cinco armonicos da el maximo

    for k in range(kmin, kmax + 1):
        suma = 0.0
        for m in range(1, 6):      # primeros 5 armónicos: k,2k,3k,4k,5k
            idx = m * k
            if idx < N // 2:       # nos quedamos en la parte positiva del espectro
                suma += absX[idx]
        if suma > max_absX:
            max_absX = suma
            k_best = k

    # Calcula la frecuencia fundamental a partir de k_best
    freq_fund = (k_best / N) * fs

    return freq_fund



from scipy import signal
from scipy.signal import correlation_lags

def determina_frecuencia_fund_xcorr(x: np.ndarray, fmin: float, fmax: float, fs: float) -> float:
    ''' Devuelve la frecuencia fundamental de un sonido musical en Hz. empleando autocorrelación'''
    ffund = None  # resultado final

    # determinar los valores mmin y mmax
    # Nota: fmax -> mmin, fmin -> mmax (relación inversa)
    mmin = int(round(fs / fmax))
    mmax = int(round(fs / fmin))

    # Calcula la autocorrelacion de x y el vector m
    xcorr = signal.correlate(x, x, mode='full')
    m = correlation_lags(len(x), len(x), mode='full')

    # Buscamos el máximo de xcorr SOLO entre mmin y mmax
    # Primero buscamos los índices reales donde m está entre esos valores
    idx_range = np.where((m >= mmin) & (m <= mmax))[0]

    # Índice (dentro del rango) del máximo de la autocorrelación
    best_local = np.argmax(xcorr[idx_range])

    # Índice real sobre el vector completo
    m_res = m[idx_range[best_local]]

    # Convierte el valor de m_res a frecuencia fundamental
    ffund = fs / m_res

    return ffund


def afinador_guitarra(x, fs):

    f_cuerdas = np.array([329.63, 246.94, 196.00, 146.8, 102.80, 82.41])

    fmin = 60
    fmax = 360

    # Determina frecuencia fundamental con la función determina_frecuencia_fund
    f_fund = determina_frecuencia_fund(x, fmin, fmax, fs)

    # Busca la cuerda más cercana EN VALOR ABS
    # NO UTILICES BUCLES: usa np.argmin sobre |f_fund - f_cuerdas|
    diferencias = np.abs(f_fund - f_cuerdas)
    cuerda_idx = np.argmin(diferencias)           # índice 0–5
    error_freq = f_fund - f_cuerdas[cuerda_idx]   # puede ser + o -

    cuerda_idx = cuerda_idx + 1   # para que sea número de cuerda 1–6

    return cuerda_idx, error_freq
